Assist from: https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

In [ ]:
%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

import copy

import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

In [ ]:


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        #Compress Stuff
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv2b = nn.Conv2d(16, 16, 3)
        
        #Classify stuff
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        # I don't understand why this asymetric padding makes the numbers work out!
        self.pad44 = nn.ZeroPad2d((4,4,4,4))
        self.pad34 = nn.ZeroPad2d((3,4,3,4))
        self.pad33 = nn.ZeroPad2d((3,3,3,3))
        self.pad23 = nn.ZeroPad2d((2,3,2,3))
        self.pad22 = nn.ZeroPad2d((2,2,2,2))
        self.pad12 = nn.ZeroPad2d((1,2,1,2))
        self.pad11 = nn.ZeroPad2d((1,1,1,1)) #Why do I need to use this padding for decompression??
        
        #Decompress Stuff
        
        # What's diff between conv2d and convtranspose2d???
        #self.conv3 = nn.ConvTranspose2d(8, 16, 3)
        self.conv3 = nn.Conv2d(16, 16, 3)
        self.conv4 = nn.Conv2d(16, 6, 3)
        self.conv5 = nn.Conv2d(6, 1, 3)
        
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        
        #self.drop2d = nn.Dropout2d()
        self.drop = nn.Dropout()
        
    def compress(self, x):
        self.size1 = x.size()
        #x, self.ind1 = F.max_pool2d(F.relu(self.drop2d(self.conv1(self.pad1(x)))), (2, 2), return_indices = True)
        x, self.ind1 = F.max_pool2d(F.relu(self.conv1(self.pad23(x))), (2, 2), return_indices = True)
        #self.size2 = x.size()
        #print(self.ind1[0][0])
        # If the size is a square you can only specify a single number
        x, self.ind2 = F.max_pool2d(F.relu(self.conv2(x)), (2, 2), return_indices = True)
        #print(self.ind2.shape)
        return x
        
    def classify(self, x):
        # Max pooling over a (2, 2) window
        #x = self.compress(x)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.drop(self.fc1(x)))
        #x = F.relu(self.fc2(x))
        x = F.relu(self.drop(self.fc2(x)))
        x = self.fc3(x)
        return x

    def classify_full(self, x):
        x = self.compress(x)
        return self.classify(x)
    
    #def classify_full_quantize(self, x):
    #    #x = torch.tensor(np.float16(self.compress(x)))
    #    x = (self.compress(x)*256).round()/256
    #    return self.classify(x)
    
    def decompress(self, x):
        #y = self.conv3(self.pad2(x))
        #print(x.shape)
        #print(self.size2)
        #y = F.max_unpool2d(x, self.ind2, kernel_size = (2,2), output_size=torch.Size([1,8,14,14]))
        y = self.upsample(x)
        #nn.Upsample(scale_factor=2, mode='nearest')
        #print(y.shape)
        #print(x.shape)
        #x = F.relu(self.drop2d(self.conv3(self.pad2(self.upsample(x)))))
        x = F.relu(self.conv3(self.pad22(self.upsample(x))))
        x = F.relu(self.conv4(self.pad22(self.upsample(x))))
        x = F.sigmoid(self.conv5(self.pad22(x)))
        
        #print(x.shape)
        
        return x
    
    def ae_full(self, x):
        x = self.compress(x)
        #print(x.shape)
        return self.decompress(x)

    #def ae_full_quantize(self, x):
    #    x = (self.compress(x)*256).round()/256
    #    return self.decompress(x)
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    #I think 'forward' needs to be defined as the default path in a Net
    def forward(self, x):
        x = self.compress(x)
        return x#self.classify(x)
        

net = Net()
print(net)

## Helper Functions

In [ ]:
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters())

In [ ]:
#criterion_ae = nn.CrossEntropyLoss()
criterion_mse = nn.MSELoss()
criterion_ae = nn.BCELoss()
#Also try BCEWithLogitsLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer_ae = optim.Adam(net.parameters())

In [ ]:
def class_test():
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            #outputs = net.classify_full(images)
            
            comp = net.compress(images).detach().round()
            if comp.max() > 15: print("WARNING, max compressed int is greater than 15:", comp.max())
            
            #print(comp)
            outputs = net.classify(comp)
            #outputs = net.classify(comp)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = (correct / total)
    print('Accuracy of the network on the 10000 test images: %f %%' % (
        100 * accuracy))
    
    return(accuracy)
    

In [ ]:
#https://github.com/pytorch/examples/blob/master/super_resolution/main.py
def ae_test():
    avg_psnr = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            #outputs = net.ae_full_quantize(images)
            #outputs = net.decompress(torch.tensor(np.float32(np.float16(net.compress(images).detach()))))
            comp = net.compress(images).detach().round()
            if comp.max() > 15: print("WARNING, max compressed int is greater than 15:", comp.max())
            #comp = np.float32(comp)
            #outputs = net.decompress(comp)
            #print(comp)
            outputs = net.decompress(comp)
            #outputs = net.decompress(comp.round())
            mse = criterion_mse(outputs, images)
            psnr = 10 * np.log10(1 / mse.item())
            avg_psnr += psnr
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testloader)))

In [ ]:
# functions to show an image

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


In [ ]:
input = torch.randn(1, 1, 28, 28)
out = net.compress(input)
print(out.min())

## Do the training...

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor()])#,
     #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False, num_workers=2)


In [ ]:
trainset = torchvision.datasets.KMNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.KMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False, num_workers=2)

In [ ]:
class_test()

In [ ]:
ae_test()

In [ ]:
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images[0:4]))
# print labels
#print(' '.join('%5s' % classes[labels[j]] for j in range(4)))
print(' '.join('%5s' % str(labels[j].item()) for j in range(4)))

In [ ]:
torch.tensor(np.float16(images[0]))
#1/(2**4)

### Optimizing Batch size (1 epoch, re-initialized) --> after AE optimized:

8:    92.1%  --> 9.6%

16:   92.0%

32:   91%  --> 7.8%

64:   90.6%

128:  89% --> 17.2% --> 88% --> 13.4%

256:  84%

512:  83.4% --> 57.7% --> 88% --> 44% --> 87.2%

In [ ]:
# five epochs from scratch gives 96% accuracy.
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net.classify_full(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

    class_test()
            
print('Finished Training')

In [ ]:
class_test()

In [ ]:
#83.7  float32
#83.6  (1-byte)
#83.9  (4-bit)
#83.7  (3-bit)
#83.7  (2-bit)
#83.6  (1-bit)

100*np.mean([class_test() for _ in range(10)])

## Frozen version 
Noticably Faster!
AE (9%) --> Frozen (67.6%, 74.7%, 79.5%, 81.2%, 82.8%, 83.7%) --> Full classify (89.5%)

In [ ]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        
        #Get "frozen" compressed images
        with torch.no_grad():
            compressed_inputs = net.compress(inputs).detach()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net.classify(compressed_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0
            
    class_test()

print('Finished Training')

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images[0:4]))
print('GroundTruth: ', ' '.join('%5s' % str(labels[j].item()) for j in range(4)))

In [ ]:
outputs = net.classify_full(images)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % str(predicted[j].item())
                              for j in range(4)))

In [ ]:
class_test()

## Train Auto-encoder

In [ ]:
# Try alternating on less than a full epoch.

# five epochs from scratch gives PSNR of 22 dB

for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer_ae.zero_grad()

        # forward + backward + optimize
        outputs = net.ae_full(inputs)
        loss = criterion_ae(outputs, inputs)
        loss.backward()
        optimizer_ae.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0
            
    ae_test()

print('Finished Training')

In [ ]:
#15.7298 (same with 1-byte quant)
#15.7283 (with 4-bit)
#15.7225 (with 3-bit)
#?? (with 2-bit)
#?? (with 1-bit)



ae_test()

## Frozen AE

In [ ]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        
        targets = copy.deepcopy(inputs.detach())
        
        #Get "frozen" compressed images
        with torch.no_grad():
            compressed_inputs = net.compress(inputs.detach()).detach()

        # zero the parameter gradients
        optimizer_ae.zero_grad()

        # forward + backward + optimize
        outputs = net.decompress(compressed_inputs)
        loss = criterion_ae(outputs, targets)
        loss.backward()
        optimizer_ae.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0


print('Finished Training')

In [ ]:
# print images
imshow(torchvision.utils.make_grid(images[0:4]))

outputs_ae = net.ae_full(images[0:4])
imshow(torchvision.utils.make_grid(outputs_ae.detach()))


### Calculated PSNR (image recreation quality metric)  (random is roughly 6.4, 17.343 frozen)

In [ ]:
with tf.Session() as sess:
  print(sess.run(tf.image.psnr(images[3].numpy(), outputs_ae.detach()[3].numpy(), max_val = 1)))

In [ ]:
ae_test()

In [ ]:
class_test()

In [ ]:
outputs[0][0].min()

In [ ]:
from torchviz import make_dot

In [ ]:
net.compress(inputs).max()